# CIS6930 Week 2: Autoencoders

---

Preparation: Go to `Runtime > Change runtime type` and choose `GPU` for the hardware accelerator.



## A magic command to check your assigned GPU

In [ ]:
gpu_info = !nvidia-smi -L
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
  print("Not connected to a GPU")
else:
  print(gpu_info)

## Libraries

In [ ]:
import copy
import random
from time import time
from typing import Any, Dict

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

## MNIST dataset from Torchvision

In [ ]:
import torchvision

train_dataset = torchvision.datasets.MNIST("./data", train=True, download=True,
                                            transform=torchvision.transforms.Compose(
                                                [torchvision.transforms.ToTensor(),
                                                 # For standardization: 0.1307 (mean), 0.3081 (var)
                                                 torchvision.transforms.Normalize((0.1307,), (0.3081,))]))
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000],
                                                             generator=torch.Generator().manual_seed(5))
test_dataset = torchvision.datasets.MNIST("/data", train=False, download=True,
                                           transform=torchvision.transforms.Compose(
                                               [torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(train_dataset[0][0].squeeze(0), cmap="gray", interpolation="none")

In [ ]:
train_dataset[0][0].view(-1).shape

## Training framework

It must have been cumbersome to copy and paste the training code to run experiments with different configurations. 

TBD in a little bit more organized manner. 


In [ ]:
def train(model: nn.Module,
              train_dataset: Dataset,
              valid_dataset: Dataset,
              config: Dict[str, Any],
              random_seed: int = 0):
  
    # Random Seeds ===============
    torch.manual_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    # Random Seeds ===============

    # GPU configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #device = torch.device("tpu" if torch.cuda.is_available() else "cpu")

    dl_train = DataLoader(train_dataset,
                          batch_size=config["batch_size"])
    dl_valid = DataLoader(valid_dataset)
                  
    # Model, Optimzier, Loss function
    model = model.to(device)

    # Optimizer
    optimizer = config["optimizer_cls"](model.parameters(), lr=config["lr"])
    loss_fn = nn.MSELoss()

    # For each epoch
    eval_list = []
    t0 = time()
    best_val = None
    best_model = None
    for n in range(config["n_epochs"]):
        t1 = time()
        print("Epoch {}".format(n))
        # Training
        train_loss = 0.
        train_pred_list = []
        train_true_list = []
        model.train()  # Switch to the training mode

        # For each batch
        for batch in dl_train:
            optimizer.zero_grad()              # Initialize gradient information
            X, y = batch
            X = X.view(X.size(0), -1).to(device) # (batch_size, 1, 28, 28) -> (batch_size, 768)
            out = model(X)                     # Call `forward()` function of the model
            loss = loss_fn(out, X)             # Calculate loss 
            loss.backward()                    # Backpropagate the loss value
            optimizer.step()                   # Update the parameters
            train_loss += loss.data.item() * config["batch_size"]

        train_loss /= (len(dl_train) * config["batch_size"])
        print("    Training loss: {:.4f}".format(train_loss))

        # Validation
        valid_loss = 0.
        valid_pred_list = []
        valid_true_list = []

        model.eval()  # Switch to the evaluation mode
        for i, batch in enumerate(dl_valid):
            X, y = batch
            X = X.view(X.size(0), -1).to(device) # (batch_size, 1, 28, 28) -> (batch_size, 768)
            out = model(X)
            loss = loss_fn(out, X.to(device))
            valid_loss += loss.data.item()

        valid_loss /= len(dl_valid)
        print("  Validation loss: {:.4f}".format(valid_loss))

        # Model selection
        if best_val is None or valid_loss < best_val:
            best_model = copy.deepcopy(model)
            best_val = valid_loss

        # Orig/generated image pair
        x_pair = torch.cat([X[0].reshape(28, 28), torch.zeros(28, 1).to(device), model(X[0]).reshape(28, 28)], axis=1)
        plt.imshow(x_pair.detach().cpu().numpy(),
                  cmap="gray", interpolation="none")
        plt.show()

        t2 = time()
        print("     Elapsed time: {:.1f} [sec]".format(t2 - t1))

        # Store train/validation loss values
        eval_list.append([n, train_loss, valid_loss, t2 - t1])

    eval_df = pd.DataFrame(eval_list, columns=["epoch", "train_loss", "valid_loss", "time"])
    eval_df.set_index("epoch")

    print("Total time: {:.1f} [sec]".format(t2 - t0))

    # Return the best model and trainining/validation information
    return {"model": best_model,
            "best_val": best_val,
            "eval_df": eval_df}

# DO NOT EDIT THE CODE UNTIL HERE

## Models (In-class exercise)

### Exercise 1: Autoencoder

In [ ]:
## Complete the code ##
class AutoEncoder(nn.Module):
    def __init__(self,
                 hidden_dim: int = 64):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(784, hidden_dim),
                                     nn.ReLU())
        self.decoder = nn.Sequential(nn.Linear(hidden_dim, 784))
        
    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def forward(self, x):
        out = self.encode(x)
        out = self.decode(out)
        return out

### Exercise 2: Denoising Autoencoder

In [ ]:
## Complete the code ##
class DenoisingAutoEncoder(nn.Module):
    def __init__(self,
                 hidden_dim: int = 64,
                 noise_factor: float = 0.01):
        super().__init__()
        #
        #
        #

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def forward(self, x):
        if self.training: # True if model.train(); False if model.eval()
            # Add noise
            
        # COMPLETE CODE (3-5 LINES)

In [ ]:
class DenoisingAutoEncoder(nn.Module):
    def __init__(self,
                 hidden_dim: int = 64,
                 noise_factor: float = 0.01):
        super().__init__()
        self.noise_factor = noise_factor
        self.encoder = nn.Sequential(nn.Linear(28 * 28, hidden_dim),
                                     nn.ReLU())
        self.decoder = nn.Sequential(nn.Linear(hidden_dim, 28 * 28))
        
    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def forward(self, x):
        if self.training: # True if model.train(); False if model.eval()
           # Add random noise for training 
           x += torch.randn_like(x) * self.noise_factor
        embedding = self.encoder(x)
        out = self.decoder(embedding)
        return out

In [ ]:
# torch.randn_like(torch.Tensor([1,2,3]))

## Experiment: Original Image Reconstruction 



In [ ]:
# Change the learning rate and run the same experiment
config = {"optimizer_cls": optim.Adam,
          "lr": 0.0001,  # lr = {0.01, 0.001, 0.0001}
          "batch_size": 16,
          "n_epochs": 5}
model = AutoEncoder()
output = train(model, train_dataset, valid_dataset, config)

### Generating images

After training, you can generate images from latent vectors

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

plt.imshow(test_dataset[0][0].squeeze(0),
           cmap="gray",
           interpolation="none")
plt.show()

# Use the first data
X, y = test_dataset[0]
z = model.encode(X.view(-1).to(device))

out0 = model.decode(z)
out1 = model.decode(z + (torch.randn_like(z) * 0.1)) # Add small noise
out2 = model.decode(z + (torch.randn_like(z) * 0.5)) # Add medium noise
out3 = model.decode(z + (torch.randn_like(z) * 1.0)) # Add large noise

plt.imshow(out0.view(28, 28).detach().cpu(),
           cmap="gray",
           interpolation="none")
plt.show()
plt.imshow(out1.view(28, 28).detach().cpu(),
           cmap="gray",
           interpolation="none")
plt.show()
plt.imshow(out2.view(28, 28).detach().cpu(),
           cmap="gray",
           interpolation="none")
plt.show()
plt.imshow(out3.view(28, 28).detach().cpu(),
           cmap="gray",
           interpolation="none")
plt.show()

In [ ]:
# Generating images from random noise
for i in range(6):
  plt.imshow(
      model(torch.randn_like(X.view(-1)).to(device)).detach().cpu().view(28, 28),
      cmap="gray",
      interpolation="none")
  plt.show()